# Evaluating a NeMo checkpoint with lm-eval

This notebook showcases how to evaluate a model with NeMo 2.0. It will guide you through the process of in-framework deployment, and evaluation of completions and chat endpoints.

In this tutorial we will evaluate an LLM on the [MMLU benchmark](https://arxiv.org/abs/2009.03300).
The benchmark measures a language model's general knowledge across 57 diverse subjects, ranging from humanities and social sciences to STEM and professional fields, using multiple-choice questions.
We will use two variants of the benchmarks: a more general one, that can be used to evaluate both base and instruction-tuned models, and a chat variant, that requires instruction-following capabilities from the model.

> NOTE: It is recommended to run this notebook inside a [NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo) which has all the required dependencies.

In [ ]:
import requests
import signal
import subprocess

from nemo.collections.llm import api
from nemo.collections.llm.evaluation.api import EvaluationConfig, EvaluationTarget
from nemo.collections.llm.evaluation.base import wait_for_fastapi_server
from nemo.utils import logging

logging.setLevel(logging.INFO)

## 1. Deploying the model

First, you need to prepare a NeMo 2 checkpoint of the model you would like to evaluate. For the purpose of this tutorial, we will use Llama 3.2 1B Instruct checkpoint, which you can download from the [NGC Catalog](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/llama-3_2-1b-instruct). Make sure to mount the directory containing the checkpoint when starting the container. In this tutorial, we assume that the checkpoint is available under `"/checkpoints/llama-3_2-1b-instruct_v2.0"` path.

> NOTE: Some steps in this tutorial are **only available for instruction-tuned (chat) models**. If you are working with a base model instead, you can still evaluate it using the `completions` endpoint and the standard `mmlu` task.

In [ ]:
# modify this variable to point to your checkpoint
CHECKPOINT_PATH = "/checkpoints/llama-3_2-1b-instruct_v2.0"

# if you are not using NeMo FW container, modify this path to point to scripts directory
SCRIPTS_PATH = "/opt/NeMo/scripts"

# modify this path if you would like to save results in a different directory
WORKSPACE = "/workspace"

After downloading the model, we can deploy it for evaluation.
The command below will start a server for the provided checkpoint in a separate process using the `deploy_in_fw_oai_server_eval.py` script.
The script will deploy the model using the [Triton Inference Server](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/index.html) and set up an OpenAI-like endpoints for querying it.

If you would like to use multiple devices for the deployment, you can specify deployment parameters to distribute your model:

In [ ]:
deploy_script = f"{SCRIPTS_PATH}/deploy/nlp/deploy_in_fw_oai_server_eval.py"

In [ ]:
!python {deploy_script} --help

In [ ]:
deploy_process = subprocess.Popen(
    ['python', deploy_script, '--nemo_checkpoint', CHECKPOINT_PATH],
)

The server exposes three endpoints:
* `/v1/triton_health`
* `/v1/completions/`
* `/v1/chat/completions/`

The `/v1/triton_health` allows you to check if the underlying Triton server is ready.
The `/v1/completions/` endpoint allows you to send prompt to the model as-is, without applying the chat template. The model responds with a text completion.
Finally, the `/v1/chat/completions/` endpoint allows for multi-turn conversational interactions with the model. This endpoint accepts a structured list of messages with different roles (system, user, assistant) to maintain context and generates chat-like responses. Under the hood, a chat template is applied to turn the conversation into a single input string.

**Please note that the chat endpoint will not work correctly for base models, as they do not define a chat template.**

In [ ]:
base_url = "http://0.0.0.0:8886"
model_name = "triton_model"

completions_url = f"{base_url}/v1/completions/"
chat_url = f"{base_url}/v1/chat/completions/"

Deployment can take a couple of minutes, especially for larger models. We will check the server status and wait until it is ready:

In [ ]:
wait_for_fastapi_server(base_url)

After the model was deployed we can query it:

In [ ]:
completions_payload = {
    "prompt": "My name is",
    "model": model_name,
    "max_tokens": 16,
}

response = requests.post(completions_url, json=completions_payload)
print(response.content.decode())

If you are working with a instruction-tuned model, you can also use the chat endpoint:

In [ ]:
chat_payload = {
    "messages": [
        {"role": "user", "content": "What is your name?"}
    ],
    "model": model_name,
    "max_tokens": 64,
}

response = requests.post(chat_url, json=chat_payload)
print(response.content.decode())

## 2. Evaluating the completions endpoint

Now, we are ready to start the evaluation. First, we will evaluate the completions endpoint on the `mmlu` task.
We will load a pre-defined configuration from [NVIDIA Evals Factory](https://pypi.org/project/nvidia-lm-eval/) lm-evaluation-harness.
This configuration has a `--num_fewshot 5` flag specified, which means that each question to the model is prepended with five examples of question-answer pairs.
This way, the model is guided on the correct way to format the output.

For the purpose of this tutorial, we will only use one sample from each subset (by setting the `limit_samples` flag to 1).
To run the full evaluation, remove this parameter from the command below.
Alternatively, you can set the parameter to, for example, 0.1 to run the evaluation on 10% of the dataset.

For more details on arguments in the EvaluationTarget and EvaluationConfig classes for evaluation, refer to [`nemo/collections/llm/evaluation/api.py`](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/evaluation/api.py)

In [ ]:
target_config = EvaluationTarget(api_endpoint={"url": completions_url, "type": "completions"})
eval_config = EvaluationConfig(
    type="mmlu",
    params={"limit_samples": 1},
    output_dir=f"{WORKSPACE}/mmlu",
)

completions_results = api.evaluate(target_cfg=target_config, eval_cfg=eval_config)

## 3. Evaluating the chat endpoint

Now, we will use a "chat" variant of the same benchmark, `mmlu_instruct`, for evaluating the chat endpoint.
In this evaluation scenario, we do not send examples of questions and answers (0-shot setting) but instead provide an instruction to the model on how the output should be formatted.

This variant of the benchmark is more challenging as it requires the model to not only provide the correct answer but also to format it according to the instruction.

Again, we will only use one sample from each subset.
You can modify this behavior by changing or removing the `limit_samples` parameter from the command below.

In [ ]:
target_config = EvaluationTarget(api_endpoint={"url": chat_url, "type": "chat"})
eval_config = EvaluationConfig(
    type="mmlu_instruct",
    params={"limit_samples": 1},
    output_dir=f"{WORKSPACE}/mmlu_instruct",
)

chat_results = api.evaluate(target_cfg=target_config, eval_cfg=eval_config)

# 4. Inspecting the results and shuting the server down

After the evaluation is finished, we can take a look at the results.
We can compare the aggregated metrics or examine the scores for particular subtasks.

It is often the case that results for the "instruct" variant are lower, as it requires strong instruction-following abilities from the model.

In [ ]:
completions_results["groups"]["mmlu_str"]["metrics"]

In [ ]:
chat_results["groups"]["mmlu_str"]["metrics"]

In [ ]:
completions_results["tasks"]["mmlu_str_professional_medicine"]["metrics"]

In [ ]:
chat_results["tasks"]["mmlu_str_professional_medicine"]["metrics"]

We can also examine the artifacts produced by both jobs.
Inside the output directories, we can find `run_config.yml` files, which store details about the evaluation setup; `lm_cache_rank0.db`, file which contains a cache that can be used to resume an interrupted evaluation; and `triton_model` directory, which holds saved metrics as well as detailed logs for each input sample and its corresponding response.

In [ ]:
! ls {WORKSPACE}/mmlu*

Finally we can close the model's server.
It can be done by sending `SIGINT` signal to the deployment process.

In [ ]:
deploy_process.send_signal(signal.SIGINT)